In [ ]:
from typing import Any, Tuple
import yaml
import numpy as np
from copy import deepcopy
from functools import reduce




6

In [2]:
def sweep_constructor(
    loader: yaml.SafeLoader, node: yaml.nodes.MappingNode
) -> dict[str, Any]:
    values = loader.construct_sequence(node.value[0][1])
    return {"type": "sweep", "values": values}


def coupled_sweep_constructor(
    loader: yaml.SafeLoader, node: yaml.nodes.MappingNode
) -> dict[str, Any]:
    target = node.value[0][1].value
    values = loader.construct_sequence(node.value[1][1])
    return {"target": target, "values": values, "type": "coupled-sweep"}


def get_loader():
    loader = yaml.SafeLoader
    loader.add_constructor("!sweep", sweep_constructor)
    loader.add_constructor("!coupled-sweep", coupled_sweep_constructor)
    return loader

In [3]:
cfg = yaml.load(open("./config.yaml"), Loader=get_loader())

In [4]:
cfg

{'sweepname': {'type': 'sweep', 'values': [1, 2, 3, 4, 5]},
 'othersweep': {'type': 'sweep', 'values': ['x', 'y', 'z']},
 'coupled_sweep': {'target': 'sweepname',
  'values': [6, 7, 8, 9, 10],
  'type': 'coupled-sweep'},
 'nested': {'a': {'b': {'type': 'sweep', 'values': [11, 12, 13, 14, 15]},
   'c': 'x'}},
 'd': {'target': 'othersweep',
  'values': ['a', 'b', 'c'],
  'type': 'coupled-sweep'}}

In [ ]:
class Run:
    def __init__(self, config: dict):
        self.config = config
        self.run_configs = []
        sweep_targets = {}
        coupled_targets = {}
        self._cfg_helper(self.config, sweep_targets, coupled_targets)

        print(sweep_targets)
        print(coupled_targets)

        self.run_configs = self._construct_run_configs(sweep_targets, coupled_targets)
        num_elems = reduce(
            lambda x, y: x * y, [len(v["values"]) for _, v in sweep_targets.items()], 1
        )
        print(
            len(self.run_configs),
            num_elems,
            len(set([tuple(s) for s in self.run_configs])),
        )

    def _cfg_helper(
        self, cfg_node: dict, sweep_targets, coupled_targets
    ) -> Tuple[list[Any], list[Any]]:
        for key, node in cfg_node.items():
            if isinstance(node, dict) and "type" in node:
                if node["type"] == "sweep":
                    sweep_targets[key] = {"key": key, "values": node["values"]}
                elif node["type"] == "coupled-sweep":
                    coupled_targets[key] = {
                        "target": node["target"],
                        "values": node["values"],
                    }
            elif isinstance(node, dict):
                self._cfg_helper(node, sweep_targets, coupled_targets)
            else:
                continue  # we ignore leafs

        for k, v in coupled_targets.items():
            sweep_targets[v["target"]]["partner"] = v["values"]

        return sweep_targets

    def _element_helper(
        self, elements, current_list, possible_partner, all_lists, *args, i=0
    ):
        i += 1
        for k in range(len(current_list)):
            v = current_list[k]
            if possible_partner is not None:
                w = possible_partner[k]
                if i < len(all_lists):
                    self._element_helper(
                        elements,
                        all_lists[i][0],
                        all_lists[i][1],
                        all_lists,
                        *args,
                        v,
                        w,
                        i=i,
                    )
                else:
                    self._element_helper(
                        elements,
                        all_lists[i][0],
                        all_lists[i][1],
                        all_lists,
                        *args,
                        v,
                        i=i,
                    )
            else:
                elements.append([*args, v])

    def _construct_run_configs(self, sweep_targets, coupled_targets):
        # make list of tuple values
        keys = list(sweep_targets.keys())
        lists = [
            (
                sweep_targets[k]["values"],
                sweep_targets[k]["partner"] if "partner" in sweep_targets[k] else None,
            )
            for k in keys
        ]
        i = 0
        elements = []
        self._element_helper(elements, lists[i][0], lists[i][1], lists, i=i)
        print(elements)

        return elements

In [103]:
runner = Run(cfg)

{'sweepname': {'key': 'sweepname', 'values': [1, 2, 3, 4, 5], 'partner': [6, 7, 8, 9, 10]}, 'othersweep': {'key': 'othersweep', 'values': ['x', 'y', 'z'], 'partner': ['a', 'b', 'c']}, 'b': {'key': 'b', 'values': [11, 12, 13, 14, 15]}}
{'coupled_sweep': {'target': 'sweepname', 'values': [6, 7, 8, 9, 10]}, 'd': {'target': 'othersweep', 'values': ['a', 'b', 'c']}}
[[1, 6, 'x', 'a', 11], [1, 6, 'x', 'a', 12], [1, 6, 'x', 'a', 13], [1, 6, 'x', 'a', 14], [1, 6, 'x', 'a', 15], [1, 6, 'y', 'b', 11], [1, 6, 'y', 'b', 12], [1, 6, 'y', 'b', 13], [1, 6, 'y', 'b', 14], [1, 6, 'y', 'b', 15], [1, 6, 'z', 'c', 11], [1, 6, 'z', 'c', 12], [1, 6, 'z', 'c', 13], [1, 6, 'z', 'c', 14], [1, 6, 'z', 'c', 15], [2, 7, 'x', 'a', 11], [2, 7, 'x', 'a', 12], [2, 7, 'x', 'a', 13], [2, 7, 'x', 'a', 14], [2, 7, 'x', 'a', 15], [2, 7, 'y', 'b', 11], [2, 7, 'y', 'b', 12], [2, 7, 'y', 'b', 13], [2, 7, 'y', 'b', 14], [2, 7, 'y', 'b', 15], [2, 7, 'z', 'c', 11], [2, 7, 'z', 'c', 12], [2, 7, 'z', 'c', 13], [2, 7, 'z', 'c', 14